In [ ]:
!pip install -qU langchain langchain-community langchainhub langchain-openai chromadb bs4 sentence_transformers transformers
!pip install -qU bitsandbytes accelerate llama-cpp-python beautifulsoup4 faiss-cpu tavily-python "langserve[all]"
!pip install -qU duckduckgo-search

In [2]:
from google.colab import userdata
import os
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage
)
from langchain_community.chat_models.huggingface import ChatHuggingFace
from transformers import AutoTokenizer

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGSMITH')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_KEY')

# Define Embeddings vs Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
    )

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
        'include_prompt_in_result' : False,
        "return_full_text": False
    },
)

chat_model = ChatHuggingFace(llm=llm)


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# Prepare Documents

In [6]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header", 'site-content')
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits[0], splits[2], len(splits)

(Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domain, Email, AWS Hosting, Social Media Accounts \nAsset Value:N/A\nCurrent ConditionRunning\nDemand20,000,000\nAddressG13, Islamabad, Pakistan", metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'}),
 Document(page_content='Purpose of Selling:\r\nSince the 3 co-founders are now based abroad, they are actively seeking entrepreneurs in Pakistan to take charge of the platform.\r\n\r\nThe price listed is negotiable to reflect a fair and accurate value of the platform considering the value of assets and long-term profitability.', metadata={'source': 'https://flippers.pk/ads/yarti

# Create Vector Store

In [7]:
try:
  vectorstore.delete_collection()
except:
  pass

# import chromadb
# client = chromadb.Client()

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, collection_metadata={"hnsw:space": "cosine"}) # downside: dowesn't do upsert
vectorstore

In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [9]:
retriever.invoke('price of website') #retriever.get_relevant_documents('abroad')

[Document(page_content='Purpose of Selling:\r\nSince the 3 co-founders are now based abroad, they are actively seeking entrepreneurs in Pakistan to take charge of the platform.\r\n\r\nThe price listed is negotiable to reflect a fair and accurate value of the platform considering the value of assets and long-term profitability.', metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'})]

# Create Prompt

In [10]:
# not compatible with zephyr
# prompt = hub.pull("rlm/rag-prompt")

# hard-code template
# template = """
# <|system|>
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>
# <|user|>
# Question: {question}
# Context: {context} </s>
# <|assistant|>
# Answer:
# """

SYSTEM_MESSAGE = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."
USER_MESSAGE = "Question: What is Yartici? \nContext: Yartici is an online art shop. "

# use tokenizer to create template
messages = [
    {
        "role": "system",
        "content": SYSTEM_MESSAGE,
    },
    {
        "role": "user",
        "content": USER_MESSAGE},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

"<|system|>\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>\n<|user|>\nQuestion: What is Yartici? \nContext: Yartici is an online art shop. </s>\n<|assistant|>\n"

In [11]:
# can also use langchain modules

messages = [
    SystemMessage(content=SYSTEM_MESSAGE),
    HumanMessage(content=USER_MESSAGE),
]

# chat model understands the expected prompt template. Interesting because I previously thought it's dumb.
chat_model._to_chat_prompt(messages)

"<|system|>\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>\n<|user|>\nQuestion: What is Yartici? \nContext: Yartici is an online art shop. </s>\n<|assistant|>\n"

In [12]:
llm.invoke(prompt)
# llm.predict(prompt)

'Answer: Yartici is a digital marketplace where artists can sell their original artwork and art-related products directly to buyers.'

In [13]:
chat_model.invoke(prompt) # chat_model.invoke() adds <user></s><assistant>

AIMessage(content='Answer: Yartici is a digital marketplace for purchasing original artwork and art supplies directly from artists.')

In [14]:
chat_model.invoke(messages) # this method works well for chat_models

AIMessage(content='Answer: Yartici is a digital marketplace where artists can sell their original artwork and art-related products directly to buyers.')

In [15]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

messages = [
    {
        "role": "system",
        "content": SYSTEM_MESSAGE,
    },
    {
        "role": "user",
        "content": "Question: {question} \nContext: {context} "},
]

raw_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(raw_prompt)

prompt = PromptTemplate.from_template(raw_prompt)
print(prompt)

prompt.invoke({'context' : "test1", 'question' : 'test question'})

<|system|>
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>
<|user|>
Question: {question} 
Context: {context} </s>
<|assistant|>

input_variables=['context', 'question'] template="<|system|>\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>\n<|user|>\nQuestion: {question} \nContext: {context} </s>\n<|assistant|>\n"


StringPromptValue(text="<|system|>\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.</s>\n<|user|>\nQuestion: test question \nContext: test1 </s>\n<|assistant|>\n")

In [16]:
# most likely this is not possible because placeholders. Only when messages are ready/entering history.
# module_messages = [
#     SystemMessage(content=SYSTEM_MESSAGE),
#     HumanMessage(content="Question: {question} \nContext: {context} "),
# ]

tuple_messages = [
    ("system", SYSTEM_MESSAGE),
    ("human", "Question: {question} \nContext: {context} ")
]

chat_prompt = ChatPromptTemplate.from_messages(tuple_messages)
chat_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Question: {question} \nContext: {context} '))])

In [17]:
chat_prompt.invoke({'context' : "test1", 'question' : 'test question'}).to_string()
# this shows that chattemplate uses "System" instead of <system>

# because we have only rendered the template without giving it any informationr egarding the model

"System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nHuman: Question: test question \nContext: test1 "

# Define Chain

In [18]:
def format_docs(docs):
    # not using stuff_documents to create chain
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Yartici?")

"Yartici is Pakistan's leading online art marketplace, established in 2020. It operates as an e-commerce platform and currently has 0-5 employees. The business generates unknown monthly revenue and profit. Its assets include a website, domain, email, AWS hosting, and social media accounts, with an asset value of N/A. Yartici's current condition is running, and it demands a valuation of 20,000,000 PKR. Its address is G13, Islamabad, Pakistan."

In [19]:
# Make chain with chat

def format_docs(docs):
    # not using stuff_documents to create chain
    return "\n\n".join(doc.page_content for doc in docs)

rag_chat_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_prompt
    | chat_model
    | StrOutputParser()
)

rag_chat_chain.invoke("What is Yartici?")

"Yartici is Pakistan's leading online art marketplace, established in 2020. It operates as an e-commerce platform and currently has 0-5 employees. The business generates unknown monthly revenue and profit. Its assets include a website, domain, email, AWS hosting, and social media accounts, with an asset value of N/A. Yartici's current condition is running, and it demands a valuation of 20,000,000 PKR. Its address is G13, Islamabad, Pakistan."

In [20]:
# Make chain with sources

from langchain_core.runnables import RunnableParallel

doc_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | chat_prompt
    | chat_model
    | StrOutputParser()
)

rag_chain_chat_from_docs = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=doc_chain)

rag_chain_chat_from_docs.invoke("What is Yartici")

{'context': [Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domain, Email, AWS Hosting, Social Media Accounts \nAsset Value:N/A\nCurrent ConditionRunning\nDemand20,000,000\nAddressG13, Islamabad, Pakistan", metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'})],
 'question': 'What is Yartici',
 'answer': "Yartici is Pakistan's leading online art marketplace, established in 2020. It operates as an e-commerce platform for buying and selling original artwork by Pakistani artists. The business currently has 0-5 employees and generates unknown monthly revenue and profit. Its assets include a website, domain, email, AWS hosting, 

In [21]:
# streaming if possible
for chunk in rag_chain_chat_from_docs.stream("What is Yartici?"):
    print(chunk)

# doesn't work with chat_model either. Most likely, streaming has to be supported by the underlying model being invoked

{'question': 'What is Yartici?'}
{'context': [Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domain, Email, AWS Hosting, Social Media Accounts \nAsset Value:N/A\nCurrent ConditionRunning\nDemand20,000,000\nAddressG13, Islamabad, Pakistan", metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'})]}
{'answer': "Yartici is Pakistan's leading online art marketplace, established in 2020. It operates as an e-commerce platform and currently has 0-5 employees. The business generates unknown monthly revenue and profit. Its assets include a website, domain, email, AWS hosting, and social media accounts, with an asset value of N/A. Yarti

In [22]:
chat_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Question: {question} \nContext: {context} '))])

In [23]:
# can use create_stuff_documents_chain as well.

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain


# expects "input" keyword
document_chain = create_stuff_documents_chain(chat_model,
                                              ChatPromptTemplate.from_messages([
                                              ("system", SYSTEM_MESSAGE),
                                              ("human", "Question: {input} \nContext: {context} ")]
))

# invoking document chain does create proper token

retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain.invoke({"input": "What is Yartici's industry?"})

{'input': "What is Yartici's industry?",
 'context': [Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domain, Email, AWS Hosting, Social Media Accounts \nAsset Value:N/A\nCurrent ConditionRunning\nDemand20,000,000\nAddressG13, Islamabad, Pakistan", metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'})],
 'answer': "Yartici's industry is e-commerce platforms, specifically online art marketplaces. They are based in Islamabad, Pakistan and were established in 2020. The exact revenue and profit are not disclosed, but they currently have 0-5 employees and their asset value is unknown. Their current condition is running, and they 

With more and more abstraction/read-made classes, the codes gets shorter but I still don't see the point.

# Add Chat History

Different ways to incorporate chat history with RAG.

### Contextualize Question

In [82]:
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"), # placeholder because .chat_history variable will comprise of a series system/human interactions.
        ("human", "Rephrame the following question as a standalone question incorporating the context: {question}"),
    ]
)

contextualize_q_chain = contextualize_q_prompt | chat_model | StrOutputParser()

contextualize_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "How large is large?",
    }
)

# why despite being False, prompt is included?

'In the context of language models, what does the term "large" refer to in terms of size or capacity?'

Despite instructing it not to answer the question, it still answers the question. Two points:
1. The prompt template may not be perfectly compatible for reasons demonstrated earlier in the nteobook. (SOLVED: After using approporiate templating method, it is now compatible. Although this was not the issue as demonstrated below.)
2. The model is not strong enough to understand instructions. (SOLVED: Added "Standalone contextualized question:" suffix to guide the model response despite the fact that the model isn't that strong. It still may not behave as expected.)

#### Verifying with custom compatible template

In [26]:
# messages = [
#     {
#         "role": "system",
#         "content": """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.""",
#     },
#     {
#         "role": "user",
#         "content": "What does LLM stand for?"
#     },
#     {
#         "role": "assistant",
#         "content": "Large language model."
#     },
#     {
#         "role": "user",
#         "content": "{question}"
#     },
# ]

# raw_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# print(raw_prompt)

# prompt = PromptTemplate.from_template(raw_prompt)
# print(prompt)

# prompt.invoke({'question' : "What is meant by large?"})

# contextualize_q_chain_custom = prompt | llm | StrOutputParser()
# contextualize_q_chain_custom.invoke({'question' : "What is meant by large?"})

Now, that we know for sure that prompt template is fully customized to make it compatible with the LLM we are using, it still doesn't understand our instruction to NOT generate the answer.

#### Use Contextualized Question RAG

In [30]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


context_rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | chat_model
)

chat_history = []

question = "What is Yartici?"
ai_msg = context_rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What is its demand?"
context_rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='The demand for Yartici, as mentioned in the provided context, is 20,000,000 Pakistani rupees (approximately $120,000 USD). This indicates the estimated value that a potential buyer would be willing to pay for the entire business, including its assets and ongoing operations.')

Note:

Huggingface messages template is a list of dict of role/content (system, user, assistant) while the LangChain is a list SystemMessage/HumanMessages or a list of tuples (role, content) e.g. system, human, ai.

### create_history_aware_retriever

In [36]:
# Can use history_aware_retriver as well

from langchain.chains import create_history_aware_retriever, create_retrieval_chain

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(chat_model, retriever, prompt)
chat_history = [HumanMessage(content="Is Yartici available for sale?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me for how much?"
})

[Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domain, Email, AWS Hosting, Social Media Accounts \nAsset Value:N/A\nCurrent ConditionRunning\nDemand20,000,000\nAddressG13, Islamabad, Pakistan", metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'})]

In [37]:
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me for how much?"
})


{'chat_history': [HumanMessage(content='Is Yartici available for sale?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me for how much?',
 'context': [Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domain, Email, AWS Hosting, Social Media Accounts \nAsset Value:N/A\nCurrent ConditionRunning\nDemand20,000,000\nAddressG13, Islamabad, Pakistan", metadata={'source': 'https://flippers.pk/ads/yartici-com-pakistans-leading-online-art-marketplace/'})],
 'answer': "Question: How much is Yartici.com valued at for sale?\nAnswer: Yartici.com, Pakistan's leading online art marketplace, is priced at Demand 20,000,000 for sale. (Three sentences maximum)"}

### ChatMessageHistory

In [54]:
# Can use chatmessagehistory class instead of list

from langchain.memory import ChatMessageHistory

# Background:

# 1.
# originally we were using context and question in Human
# tuple_messages = [
#     ("system", SYSTEM_MESSAGE),
#     ("human", "Context: {context} \n{question} \n")
# ]
# chat_prompt = ChatPromptTemplate.from_messages(tuple_messages)

# 2.
# with chat we used placeholder like this for custom chain
# with system containing the context
# almost the same but merged question with chat history to form messages so
# don't have to treat latest question differently

# qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", qa_system_prompt),
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human", "{question}"),
#     ]
# )

# official docs suggest context in the system and a chat history including latest message in "messages"
question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Answer the user's questions based on the below context:\n\n{context}",),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat_model, question_answering_prompt)

ephemeral_chat_history = ChatMessageHistory()
ephemeral_chat_history.add_user_message("What is Yartici?")

document_chain.invoke(
    {
        "messages": ephemeral_chat_history.messages,
        "context": [Document(page_content='Yartici is Pakistan"s leading art marketplace.')],
    }
)

'Yartici is a leading online art marketplace based in Pakistan. It provides a platform for artists to showcase and sell their artwork directly to buyers, eliminating the need for intermediaries. Yartici offers a wide range of original paintings, prints, sculptures, and other art forms from both established and emerging artists in Pakistan and around the world. The platform also provides a secure and convenient way for buyers to purchase art online, with options for payment, shipping, and returns.'

In [64]:
# # not the best because the last question is not contextialized here:

# def parse_retriever_input(params):
#     return params["messages"][-1].content

# retrieval_chain = RunnablePassthrough\
#                   .assign(context=parse_retriever_input | retriever,)\
#                   .assign(answer=document_chain,)

# response = retrieval_chain.invoke(
#     {
#         "messages": ephemeral_chat_history.messages,
#     }
# )

# ephemeral_chat_history.add_ai_message(response["answer"])
# ephemeral_chat_history.add_user_message("tell me more about that!")

# retrieval_chain.invoke(
#     {
#         "messages": ephemeral_chat_history.messages,
#     },
# )

# # Alternative (short-hand) if variables need not be saved

# retrieval_chain_with_only_answer = (
#     RunnablePassthrough.assign(
#         context=parse_retriever_input | retriever,
#     )
#     | document_chain
# )


In [58]:
# with query transformation

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1, (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat_model | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")


# alternatieve for create_retrieval_chain as above
conversational_retrieval_chain = RunnablePassthrough\
                                .assign(context=query_transforming_retriever_chain,)\
                                .assign(answer=document_chain,)

In [62]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("What is Yartici?")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])
response

{'messages': [HumanMessage(content='What is Yartici?'),
  AIMessage(content="Yartici is an online art marketplace based in Pakistan. It was established in 2020 and operates as an e-commerce platform for buying and selling original artworks by Pakistani artists. The company's website, domain, email, AWS hosting, and social media accounts are its primary assets. As of now, the business has no employees, and its revenue, profit, property value, and rent are not disclosed. The current condition of the business is running, and it has a demand of $20 million. The address provided is G13, Islamabad, Pakistan.")],
 'context': [Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valueN/A\nProperty RentN/A\nAsset include:Website, Domai

In [63]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='What is Yartici?'),
  AIMessage(content="Yartici is an online art marketplace based in Pakistan. It was established in 2020 and operates as an e-commerce platform for buying and selling original artworks by Pakistani artists. The company's website, domain, email, AWS hosting, and social media accounts are its primary assets. As of now, the business has no employees, and its revenue, profit, property value, and rent are not disclosed. The current condition of the business is running, and it has a demand of $20 million. The address provided is G13, Islamabad, Pakistan."),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(page_content="Yartici.com - Pakistan's Leading Online Art Marketplace   | E-Commerce Platforms\n\n \n  G13, Islamabad, Pakistan  -\n  Islamabad\n\n\n\n\n\n\n\n\nBusiness CategoryE-Commerce Platforms\nBusiness Established Year2020\nRevenue Per MonthN/A\nMonthly ProfitN/A\nNumber of employees0 to 5\nProperty valu

Overall, this didn't make things THAT easier. I am more and more convinced that LangChain isn't THAT necessary.

### Avoid Explicit Handling of New Messages using Runnable with Memory

In [ ]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"), # placeholder because .chat_history variable will comprise of a series system/human interactions.
        ("human", "{question} "),
    ]
)

contextualize_q_chain = contextualize_q_prompt | chat_model | StrOutputParser()

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


context_rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | chat_model
)

In [89]:
from langchain_core.runnables.history import RunnableWithMessageHistory


demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    context_rag_chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="question",
    history_messages_key="chat_history",
)

chain_with_message_history.invoke(
    {"question": "What is Yartici."},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='Yartici is a Pakistani online art marketplace, established in 2020. It operates as an e-commerce platform and is currently running with a demand of 20 million Pakistani rupees. Its assets include a website, domain, email, AWS hosting, and social media accounts, but the value of these assets is not disclosed. The company has no physical property, and its revenue, profit, and number of employees are also not specified. Its location is G13, Islamabad, Pakistan.')

In [90]:
chain_with_message_history.invoke(
    {"question": "When was it established"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='Yartici was established in 2020, according to the provided context.')

In [91]:
chain_with_message_history.invoke(
    {"question": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content="I'm sorry, I'm not aware of what you asked me before. Please provide the question again, so I can assist you.")

In [92]:
chain_with_message_history.invoke(
    {"question": "Tell me where are the people who created this company?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='The provided context does not disclose the location or identities of the people who created Yartici. The only information provided is that Yartici is based in G13, Islamabad, Pakistan.')

In [93]:
chain_with_message_history.invoke(
    {"question": "Are co-founders of the company abroad"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content="The provided context does not mention anything about the co-founders of Yartici being abroad. The only information provided is that Yartici is based in G13, Islamabad, Pakistan. However, it's always possible that the founders may be living abroad, but this is not explicitly stated in the given context.")

In [94]:
chain_with_message_history.invoke(
    {"question": "Why are the cofounders selling it?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content="The provided context doesn't offer a specific reason why the co-founders of Yartici are selling the company. It only mentions that since the co-founders are now based abroad, they are actively seeking entrepreneurs in Pakistan to take charge of the platform. The price listed is negotiable to reflect a fair and accurate value of the platform considering the value of assets and long-term profitability.")

1. This just means that the model is a bit weak but the overall process is more streamlines as memory is managed automatically.
2. We can further add a custom function before chain (as part of chain) to trim messages or summarization.

In [ ]:
# cleanup
vectorstore.delete_collection()